# Large Scale Production Cost Modeling with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

This example shows a basic PCM simulation using the system data assembled in the
[US-System example](../../notebook/PowerSystems_examples/US_system.ipynb).

### Dependencies

In [1]:
using SIIPExamples
using PowerSystems
using PowerSimulations
using PowerGraphics
using Logging
pkgpath = dirname(dirname(pathof(SIIPExamples)))
PSY = PowerSystems
PSI = PowerSimulations
plotlyjs()

┌ Info: Precompiling PowerSimulations [e690365d-45e2-57bb-ac84-44ba829e73c4]
└ @ Base loading.jl:1273


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-14955074478522790727\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16247964366897930601\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-14955074478522790727\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyJSBackend()

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [2]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.1)

using Xpress
solver = optimizer_with_attributes(Xpress.Optimizer, "MIPRELSTOP" => 0.1, "OUTPUTLOG" => 1)

┌ Info: Xpress: Found license file /Users/cbarrows/Applications/fico-xpress/xpressmp/bin/xpauth.xpr
└ @ Xpress /Users/cbarrows/.julia/packages/Xpress/kcAft/src/license.jl:42
┌ Info: Xpress: Development license detected.
└ @ Xpress /Users/cbarrows/.julia/packages/Xpress/kcAft/src/license.jl:84


MathOptInterface.OptimizerWithAttributes(Xpress.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("MIPRELSTOP") => 0.1, MathOptInterface.RawParameter("OUTPUTLOG") => 1])

### Create a `System` from US data.
If you have run the
[US-System example.](../../notebook/PowerSystems_examples/US-System.ipynb), the data will
be serialized in the json and H5 format, so we can efficiently deserialize it:

In [3]:
sys = System(joinpath(pkgpath,"US-System", "SIIP", "sys.json"))

Base.IOError: IOError: open: no such file or directory (ENOENT)

### Selecting flow limited lines
Since PowerSimulations will apply constraints by component type (e.g. Line), we need to
change the component type of the lines on which we want to enforce flow limits. So, let's
change the device type of certain branches from Line to MonitoredLine differentiate
treatment when we build the model. Here, we can select inter-regional lines, or lines
above a voltage threshold.

In [4]:
for line in get_components(Line, sys)
    if (get_basevoltage(get_from(get_arc(line))) >= 230.0) &&
       (get_basevoltage(get_to(get_arc(line))) >= 230.0)
        #if get_area(get_from(get_arc(line))) != get_area(get_to(get_arc(line)))
        @info "Changing $(get_name(line)) to MonitoredLine"
        convert_component!(MonitoredLine, line, sys)
    end
end

branches = Dict{Symbol, DeviceModel}(
    :L => DeviceModel(Line, StaticLineUnbounded),
    :T => DeviceModel(Transformer2W, StaticTransformer),
    :TT => DeviceModel(TapTransformer, StaticTransformer),
    :ML => DeviceModel(MonitoredLine, StaticLine),
    :DC => DeviceModel(HVDCLine, HVDCDispatch),
)

devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalStandardUnitCommitment),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, FixedOutput),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
    :ILoads => DeviceModel(InterruptibleLoad, InterruptiblePowerLoad),
)

template = OperationsProblemTemplate(DCPPowerModel, devices, branches, Dict());

UndefVarError: UndefVarError: sys not defined

### Build and execute single step problem

In [5]:
op_problem =
    OperationsProblem(GenericOpProblem, template, sys; optimizer = solver, horizon = 24, slack_variables = false, use_parameters = true)

res =solve!(op_problem)

UndefVarError: UndefVarError: template not defined

### Analyze results

In [6]:
fuel_plot(res, sys, load = true)

UndefVarError: UndefVarError: res not defined

## Sequential Simulation
In addition to defining the formulation template, sequential simulations require
definitions for how information flows between problems.

In [7]:
sim_folder = mkpath(joinpath(pkgpath, "Texas-sim"), )
stages_definition = Dict(
    "UC" => Stage(GenericOpProblem, template, sys, solver; slack_variables = true)
)
order = Dict(1 => "UC")
horizons = Dict("UC" => 24)
intervals = Dict("UC" => (Hour(24), Consecutive()))
cache = Dict(("UC",) => TimeStatusChange(PSY.ThermalStandard, PSI.ON))
DA_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = InterStageChronology(),
    cache = cache, #needed for ThermalStandardUC not for Basic
)

UndefVarError: UndefVarError: template not defined

### Define and build a simulation

In [8]:
sim = Simulation(
    name = "Texas-test",
    steps = 10,
    stages = stages_definition,
    stages_sequence = DA_sequence,
    simulation_folder = "Texas-sim"
)
build!(sim, console_level = Logging.Info, file_level = Logging.Debug)

UndefVarError: UndefVarError: stages_definition not defined

### Execute the simulation

In [9]:
sim_results = execute!(sim)

UndefVarError: UndefVarError: sim not defined

### Load and analyze results

In [10]:
uc_results = load_simulation_results(sim_results, "UC");

fuel_plot(uc_results, sys, load = true)

UndefVarError: UndefVarError: sim_results not defined

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*